# ST7 Planification quotidienne d’une équipe mobile

In [1]:
# Modules de base
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Module relatif à Gurobi
from gurobipy import *

## Formulation du problème

### Notation
- $ V $ : Nombre de tâches + 1 (V pour vertex)
- $ T $ : Nombre de techniciens
- $ i, j \in \{0, ..., V - 1\} $ : les indices des tâches
- $ k \in \{0, ..., T - 1 \} $ : les indices des techniciens
- $ (opening_i)_i $ : l’ouverture des sites
- $ (closing_i)_i $ : la fermeture des sites
- $ (start_k)_k $ : le début de travail des employés
- $ (end_k)_k $ : la fin de travail de chaque employé
- $ (duration_i)_i $ : la durée de chaque tâche
- $ (distance_{i, j}) $ : la distance entre les sites
- $ speed $ : la vitesse de déplacement des techniciens
- $ M $ : majorant pour notre problème



### Variables de décisions
- $ \tilde{T} $ : Le nombre de techniciens actifs (avec une tâche ou plus)
- $ (x_{i, j})_{i, j} $, $ x_{i, j} = 1 $ ssi. (i, j) est un arc de i vers j
- $ (y_{k, i})_{k, i} $, $ y_{k, i} = 1 $ ssi. le technicien k effectue la tâche i
- $ (b_i)_i $, $B_i$ est le début de chaque tâche en minute (b pour beginning)


### Objectif d’optimisation
$$ min \sum_{i, j} x_{i, j} \cdot distance_{i, j} $$


### Contraintes
- (C1) : Il n’y a qu’une seule composante connexe dans le graphe des parcours
$$ \sum_{i, j} x_{i, j} = \tilde{T} + (V - 1) $$
- (C2) Il y a autant d’arcs sortant/entrant au dépôt que de techniciens actifs
$$ \tilde{T} = \sum_i x_{0, i} = \sum_i x_{i, 0}$$
- (C3) Le site de chaque tâche est visité par exactement un technicien
$$ \forall i > 0, \sum_j x_{i, j} = \sum_j x_{j, i} = 1 $$
- (C4) Deux tâches qui se suivent doivent être faites par le même technicien
$$ \forall i, j > 0, x_{i, j} \implies \forall k > 0, y_{k, i} = y_{k, j} \\
\Longleftrightarrow \forall i, j > 0, \forall k > 0, y_{k, i} \le y_{k, j} + M \cdot (1 - x_{i, j}) \text{ et } y_{k, j} \le y_{k, i} + M \cdot (1 - x_{i, j}) $$
- (C5) Lorsqu’un travail est effectué sur une tâche, la tâche doit être disponible.
$$ \forall i > 0, (b_i \ge opening_i) \land (b_i + duration_i \le closing_i) $$
- (C6) La fenêtre de temps entre deux travaux doit être suffisante pour le trajet
$$ \forall i, j > 0, x_{i, j} = 1 \implies b_i + duration_i + \frac{distance_{i, j}}{speed} \le b_j \\
\Longleftrightarrow \forall i, j > 0, b_i + duration_i + \frac{distance_{i, j}}{speed} \le b_j + M \cdot (1-x_{i, j}) $$
- (C7) Un technicien doit avoir suffisamment de temps pour aller à son premier site
$$ \forall k > 0, \forall i > 0, y_{k, i} \land x_{0, i} \implies start_k + \frac{distance_{0, i}}{speed} \le b_i \\
\Longleftrightarrow \forall k > 0, \forall i > 0, start_k + \frac{distance_{0, i}}{speed} \le b_i + (2 - y_{k, i} - x_{0, i}) \cdot M $$
- (C8) Un technicien doit avoir suffisamment de temps pour rentrer au dépot après sa dernière tâche
$$ \forall k > 0, \forall i > 0, y_{k, i} \land x_{i, 0} \implies b_i + duration_i + \frac{distance_{i, 0}}{speed} \le end_k \\
\Longleftrightarrow \forall k > 0, \forall i > 0, b_i + duration_i + \frac{distance_{i, 0}}{speed} \le end_k + (2 - y_{k, i} - x_{i, 0}) \cdot M $$
- (C9) Un technicien n’effectue que des tâches qu’il est capable d’effectuer
$$ \forall k > 0, \forall i > 0, levelTech_k \ge levelTask_i - M \cdot (1 - y_{k, i}) $$
- (C10) Une tâche est réalisée par un seul employé
$$ \forall i > 0, \sum_k y_{k, i} = 1 $$



## Variables de décisions
$ X \in R_{V + 1 \times V + 1}$

In [32]:
# module imports
from models_v1 import Employee, Task
from file_paths import path_finland

In [33]:
# reading dataframe into python objects
Employee.load_excel(path_finland)
Task.load_excel(path_finland)

In [34]:
from utils import *
def conversion(time):
    res = 0
    res += 60 * time.hour + time.minute
    return(res)


In [13]:
V = len(Employee.list)
T = len(Task.list)

In [35]:
m = Model("DB")
X = {(i, j) : m.addVar(vtype = GRB.BINARY, name = f'x{i}_{j}') for i in range(0,V) for j in range(0,V)}
Y = {(k, i) : m.addVar(vtype = GRB.BINARY, name = f'y{k}_{i}') for k in range(0,T) for i in range(0,V)}
B = {i : m.addVar(vtype = GRB.CONTINUOUS, name = f'b{i}') for i in range(0,V)}
T_actif = m.addVar(vtype = GRB.INTEGER, name = 'T_actif')

In [9]:
#C1
C1 = m.addConstr(quicksum([X[(i,j)] for i in range(0,V) for j in range(0,V)]) == T_actif + V - 1, name ="C1")

In [10]:
#C3
C3_entrée = dict()
C3_sortie = dict()
for i in range(0,V):
    C3_entrée[i] = m.addConstr(quicksum([X[(i,j)] for j in range(0,V)]) == 1, name = f'c3e{i}')
    C3_sortie[i] = m.addConstr(quicksum([X[(j,i)] for j in range(0,V)]) == 1, name = f'c3s{i}')

In [ ]:
#C5
C5 = dict()
for i in range(0,V):
    